In [350]:

# Importar las bibliotecas necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
import numpy as np

In [290]:
#Importamos bases de datos
# Especifica las irl de las bases Parquet
ruta_saldos  = 'https://datamxdevsa.blob.core.windows.net/crskmex000/DSTest/0saldos.parquet'
ruta_transferencias = 'https://datamxdevsa.blob.core.windows.net/crskmex000/DSTest/0transferencias.parquet'
ruta_clientes = 'https://datamxdevsa.blob.core.windows.net/crskmex000/DSTest/0clientes.parquet'

# Lee los archivos Parquet y carga los datos en un DataFrame de Pandas
df_transferencias = pd.read_parquet(ruta_transferencias)
df_saldos = pd.read_parquet(ruta_saldos)
df_clientes = pd.read_parquet(ruta_clientes)

In [291]:
#Seleccionamos campos 
df_saldos = df_saldos[['Contrato', 'PlanProducto', 'NroDocum',
    'SALDO_202208', 'SALDO_202209', 'SALDO_202210']]
# Agrupar por las columnas especificadas y sumar los valores
df_saldos = df_saldos.groupby(['Contrato', 'PlanProducto', 'NroDocum']).agg({
    'SALDO_202208': 'sum',
    'SALDO_202209': 'sum',
    'SALDO_202210': 'sum'
}).reset_index()

In [292]:
df_saldos

,Contrato,PlanProducto,NroDocum,SALDO_202208,SALDO_202209,SALDO_202210
0,10098,7256,1184755661,0.000000e+00,0.0,0.000000e+00
1,10103,7256,1036202838,0.000000e+00,0.0,0.000000e+00
2,10148,7256,1039603374,0.000000e+00,0.0,0.000000e+00
3,10280,7256,1115021901,0.000000e+00,0.0,0.000000e+00
4,10300,3376,1022669639,0.000000e+00,0.0,0.000000e+00
...,...,...,...,...,...,...
201495,9999884,4899,1084149110,0.000000e+00,0.0,0.000000e+00
201496,9999898,7256,1020019654,2.183404e+08,0.0,2.215017e+08
201497,9999908,7256,1019831767,0.000000e+00,0.0,0.000000e+00
201498,9999924,7256,1082986038,0.000000e+00,0.0,0.000000e+00


In [293]:
# Convertir las columnas de fecha a tipo datetime
df_transferencias['FechaEfectiva'] = pd.to_datetime(df_transferencias['FechaEfectiva'])
df_transferencias['FechaProceso'] = pd.to_datetime(df_transferencias['FechaProceso'])
# Filtrar el DataFrame
filtro_fecha = (df_transferencias['FechaEfectiva'] >= '2022-07-01') & (df_transferencias['FechaEfectiva'] <= '2022-10-31') & \
               (df_transferencias['FechaProceso'] >= '2022-07-01') & (df_transferencias['FechaProceso'] <= '2022-10-31')

df_transferencias = df_transferencias[filtro_fecha]
#Seleccionamos campos
df_transferencias = df_transferencias[['Contrato', 'PlanProducto', 'FechaProceso']]
# Agrupar por las columnas 'Contrato' y 'PlanProducto' y contar las ocurrencias
df_transferencias = df_transferencias.groupby(['Contrato', 'PlanProducto']).agg({
    'FechaProceso': 'count'  # Puedes seleccionar cualquier columna para contar las ocurrencias
}).reset_index()

# Renombrar la columna de conteo
df_transferencias = df_transferencias.rename(columns={'FechaProceso': 'NumeroTransaciones'})

In [294]:
df_transferencias

,Contrato,PlanProducto,NumeroTransaciones
0,10359,4899,4
1,11595,4899,4
2,12004,4899,4
3,12973,3376,2
4,14150,8404,4
...,...,...,...
10940,9995431,3376,1
10941,9995999,7256,2
10942,9996808,7256,4
10943,9998013,7256,6


In [295]:
# Convertir la columna FecNacim a tipo datetime
df_clientes['FecNacim'] = pd.to_datetime(df_clientes['FecNacim'])

# Calcular la diferencia entre la fecha de nacimiento y octubre de 2022
diferencia = pd.Timestamp('2022-10-31') - df_clientes['FecNacim']

# Extraer el componente de años de la diferencia
df_clientes['Edad'] = diferencia // pd.Timedelta(days=365.25)

#Seleccionamos campos
df_clientes = df_clientes[['NroDocum','Edad']]

# Eliminamos los NroDocum duplicados 
df_clientes = df_clientes.drop_duplicates(subset=['NroDocum'], keep=False)

In [296]:
df_clientes

,NroDocum,Edad
0,1092070504,67.0
1,1097627287,53.0
2,1066619616,61.0
3,1079454835,36.0
4,1199126816,41.0
...,...,...
108087,1077819263,45.0
108088,1050991361,58.0
108089,1117842187,57.0
108090,1065109336,46.0


In [352]:
# Cruza los datos
datos_completos = pd.merge(df_transferencias, df_saldos, on=['Contrato', 'PlanProducto'], how = 'left')
datos_completos = pd.merge(datos_completos, df_clientes, on='NroDocum', how = 'left')

In [353]:
datos_completos

,Contrato,PlanProducto,NumeroTransaciones,NroDocum,SALDO_202208,SALDO_202209,SALDO_202210,Edad
0,10359,4899,4,1050921348,2.548215e+07,0.0,6.803576e+05,37.0
1,11595,4899,4,1047147717,9.237657e+08,0.0,9.641952e+08,41.0
2,12004,4899,4,1055321504,1.199656e+08,0.0,1.342263e+08,38.0
3,12973,3376,2,1096057493,1.774264e+07,0.0,1.626836e+07,57.0
4,14150,8404,4,1104760101,9.335649e+08,0.0,9.192543e+08,48.0
...,...,...,...,...,...,...,...,...
10941,9995431,3376,1,1169386237,2.175818e+07,0.0,2.196443e+07,60.0
10942,9995999,7256,2,1120871836,1.681396e+06,0.0,1.705437e+06,45.0
10943,9996808,7256,4,1040127497,1.419557e+07,0.0,1.430835e+07,40.0
10944,9998013,7256,6,1049741566,1.168152e+08,0.0,1.219959e+08,33.0


In [361]:
# Agrupar por las columnas especificadas y sumar los valores
datos_completos = datos_completos.groupby(['NroDocum']).agg({
    'Contrato': 'count', 
    'PlanProducto': 'count', 
    'SALDO_202208': 'sum',
    'SALDO_202209': 'sum',
    'SALDO_202210': 'sum',
    'NumeroTransaciones': 'sum',
    'Edad': 'max'
}).reset_index()
# Crear una etiqueta para indicar si el cliente retiró el 70% o más de su saldo 
datos_completos['saldo_promedio'] = (datos_completos['SALDO_202210'] + datos_completos['SALDO_202209'] + datos_completos['SALDO_202208'])/3
datos_completos['retiro_70'] = np.where(datos_completos['SALDO_202210'] <= 0.3 * datos_completos['SALDO_202208'], 1, 0)

In [362]:
datos_completos = datos_completos.dropna()

In [363]:
datos_completos 

,NroDocum,Contrato,PlanProducto,SALDO_202208,SALDO_202209,SALDO_202210,NumeroTransaciones,Edad,saldo_promedio,retiro_70
0,1000001634,1,1,4.914860e+07,0.0,4.967498e+07,4,53.0,3.294119e+07,0
1,1000007479,1,1,5.096480e+07,0.0,2.548386e+08,6,62.0,1.019345e+08,0
2,1000014510,1,1,3.586996e+07,0.0,3.263579e+07,1,47.0,2.283525e+07,0
3,1000031391,1,1,8.569985e+07,0.0,1.437901e+08,1,37.0,7.649664e+07,0
4,1000065264,1,1,5.693606e+07,0.0,5.850999e+07,1,52.0,3.848202e+07,0
...,...,...,...,...,...,...,...,...,...,...
9972,1199857419,1,1,2.981509e+08,0.0,2.925139e+08,7,45.0,1.968883e+08,0
9973,1199865441,1,1,3.771996e+06,0.0,4.656868e+06,4,59.0,2.809621e+06,0
9974,1199931591,1,1,1.777920e+08,0.0,1.403500e+08,2,58.0,1.060473e+08,0
9975,1199935297,1,1,1.454326e+08,0.0,1.708800e+08,8,57.0,1.054375e+08,0


In [364]:
# Seleccionar las características relevantes y el objetivo
feature_cols = ['SALDO_202208','NumeroTransaciones','Edad']
X = datos_completos[feature_cols]
y = datos_completos['retiro_70']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar el modelo de regresión logística
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_proba = lr.predict_proba(X_test_scaled)[:, 1]

# Evaluar el rendimiento del modelo
auc = roc_auc_score(y_test, y_pred_proba)
print("El área bajo la curva (AUC) en el conjunto de prueba es:", auc)

El área bajo la curva (AUC) en el conjunto de prueba es: 0.7300239163560354


In [365]:


# Suponiendo que ya tienes y_test y y_pred

# Convertir las probabilidades predichas en clases
# Si y_pred es una probabilidad continua, debes convertirla en clases binarias
# Por ejemplo, si y_pred > 0.5, lo consideramos como clase 1, de lo contrario, clase 0
y_pred_binary = np.where(y_pred_proba > 0.5, 1, 0)

# Generar la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_binary)

# Imprimir la matriz de confusión
print("Matriz de Confusión:")
print(conf_matrix)

Matriz de Confusión:
[[2613    0]
 [ 381    0]]


In [366]:
# Calcular precision, recall y f1-score
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

# Obtener el reporte de clasificación
report = classification_report(y_test, y_pred_binary)

# Imprimir las métricas
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Reporte de Clasificación:\n", report)

Precision: 0.0
Recall: 0.0
F1-score: 0.0
Reporte de Clasificación:
               precision    recall  f1-score   support

           0       0.87      1.00      0.93      2613
           1       0.00      0.00      0.00       381

    accuracy                           0.87      2994
   macro avg       0.44      0.50      0.47      2994
weighted avg       0.76      0.87      0.81      2994



C:\Users\Alex Nolasco\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alex Nolasco\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alex Nolasco\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alex Nolasco\anaconda3\Lib\site-packages\sklea

In [360]:
# Entrenar el modelo de regresión logística con statsmodels
logit_model = sm.Logit(y, X)
result = logit_model.fit()

# Imprimir resumen de los parámetros del modelo
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.356300
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:              retiro_70   No. Observations:                 9977
Model:                          Logit   Df Residuals:                     9974
Method:                           MLE   Df Model:                            2
Date:                Sun, 10 Mar 2024   Pseudo R-squ.:                 0.07328
Time:                        20:58:17   Log-Likelihood:                -3554.8
converged:                       True   LL-Null:                       -3835.9
Covariance Type:            nonrobust   LLR p-value:                8.281e-123
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
SALDO_202208       -4.708e-09   3.59e-10    -13.126      0.000   -5.41e-09      -4e-09
Numer

C:\Users\Alex Nolasco\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\Alex Nolasco\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
